In [1]:
import pandas as pd

In [2]:
def load_dataset_ddi(path, effect):
    asymmetric_ddi = pd.read_csv(path, delimiter=",")
    asymmetric_ddi.loc[asymmetric_ddi[effect].isin(
        ['Excretion_rate', 'Excretory_function', 'excretion_rate', 'excretion rate']), effect] = 'excretion'
    asymmetric_ddi.loc[asymmetric_ddi[effect].isin(['Process_of_absorption', 'Absorption']),
                       effect] = 'absorption'
    asymmetric_ddi.loc[asymmetric_ddi[effect].isin(
        ['Serum_concentration', 'Serum_concentration_of', 'Serum_level', 'serum level', 'serum_concentration',
         'serum concentration', 'serum concentration of ']), effect] = 'serum_concentration'
    asymmetric_ddi.loc[asymmetric_ddi[effect].isin(['Metabolism', 'metabolism']), effect] = 'metabolism'

    asymmetric_ddi = asymmetric_ddi.loc[
        asymmetric_ddi[effect].isin(['excretion', 'absorption', 'serum_concentration', 'metabolism'])]

#     asymmetric_ddi = asymmetric_ddi[
#         ['precipitantDrugLabel', 'objectDrugLabel', 'Effect', 'Impact', 'precipitantDrug', 'objectDrug']]

    asymmetric_ddi.loc[asymmetric_ddi.Impact.isin(
        ['Increase', 'Higher', 'Worsening', 'higher', 'increase', 'worsening']), 'Impact'] = 'increase'
    asymmetric_ddi.loc[asymmetric_ddi.Impact.isin(['Decrease', 'Lower', 'Reduced', 'Reduction']),
                       'Impact'] = 'decrease'

#     asymmetric_ddi = asymmetric_ddi.dropna()
    asymmetric_ddi.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    return asymmetric_ddi

def get_ddi_triples(pk_ddi):
    set_drugs = set()
    graph_ttl = """@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
    @prefix ex: <http://example/#> .
    @prefix dpi: <http://example/DrugProteinInteraction#> .
    @prefix ddi: <http://example/DrugDrugInteraction#> ."""
    save_ttl_file(graph_ttl, 'PK_DDI_DPI.ttl')
    for index, row in pk_ddi.iterrows():
        graph_ttl = ''
        if row['precipitantDrug'] not in set_drugs:
            graph_ttl = graph_ttl + """<http://example/Drug/""" + row['precipitantDrug'] + """>\trdf:type\tex:Drug .\n"""
            #graph_ttl = graph_ttl + """<http://example/Drug/""" + row['precipitantDrug'] + """>\trdf:hasName\t""" + row['precipitantDrugLabel'] + """ .\n"""
            #graph_ttl = graph_ttl + """<http://example/Drug/""" + row['precipitantDrug'] + """>\trdf:hasID\t""" + row['precipitantDrug'] + """ .\n"""
            set_drugs.add(row['precipitantDrug'])
        if row['objectDrug'] not in set_drugs:
            graph_ttl = graph_ttl + """<http://example/Drug/""" + row['objectDrug'] + """>\trdf:type\tex:Drug .\n"""
            #graph_ttl = graph_ttl + """<http://example/Drug/""" + row['objectDrug'] + """>\trdf:hasName\t""" + row['objectDrugLabel'] + """ .\n"""
            #graph_ttl = graph_ttl + """<http://example/Drug/""" + row['objectDrug'] + """>\trdf:hasID\t""" + row['objectDrug'] + """ .\n"""
            set_drugs.add(row['objectDrug'])
        
        graph_ttl = graph_ttl + """ddi:""" + row['precipitantDrug'] + row[
            'objectDrug'] + row['Effect'] + """\trdf:type\tex:DDI .\n"""
        
        graph_ttl = graph_ttl + """ddi:""" + row['precipitantDrug'] + row[
            'objectDrug'] + row['Effect'] + """\tex:Effect\tex:""" + row['Effect'] + """ .\n"""
        graph_ttl = graph_ttl + """ddi:""" + row['precipitantDrug'] + row[
            'objectDrug'] + row['Effect'] + """\tex:Impact\tex:""" + row['Impact'] + """ .\n"""

        graph_ttl = graph_ttl + """ddi:""" + row['precipitantDrug'] + row[
            'objectDrug'] + row['Effect'] + """\tex:precipitant_drug\t<http://example/Drug/""" + row['precipitantDrug'] + """> .\n"""
        graph_ttl = graph_ttl + """ddi:""" + row['precipitantDrug'] + row[
            'objectDrug'] + row['Effect'] + """\tex:object_drug\t<http://example/Drug/""" + row['objectDrug'] + """> .\n"""
        save_ttl_file(graph_ttl, 'PK_DDI_DPI.ttl')
    return set_drugs

def get_dpi_triples(dpi, set_drugs):
    set_protein = set()
    for index, row in dpi.iterrows():
        #organism = row['organism'].replace(' ', '_')
        graph_ttl = ''
        if row['drugbank_id'] not in set_drugs:
            graph_ttl = graph_ttl + """<http://example/Drug/""" + row['drugbank_id'] + """>\trdf:type\tex:Drug .\n"""
            #graph_ttl = graph_ttl + """<http://example/Drug/""" + row['drugbank_id'] + """>\trdf:hasID\t""" + row['drugbank_id'] + """ .\n"""
            set_drugs.add(row['drugbank_id'])
        if row['uniprot_id'] not in set_protein:
            graph_ttl = graph_ttl + """<http://example/Protein/""" + row['uniprot_id'] + """>\trdf:type\tex:Protein .\n"""
            set_protein.add(row['uniprot_id'])
        
        graph_ttl = graph_ttl + """dpi:""" + row['drugbank_id'] + row[
            'uniprot_id'] +  """\trdf:type\tex:DPI .\n"""
        
        graph_ttl = graph_ttl + """dpi:""" + row['drugbank_id'] + row[
            'uniprot_id'] +  """\tex:hasProtein\t<http://example/Protein/""" + row['uniprot_id'] + """> .\n"""
        graph_ttl = graph_ttl + """dpi:""" + row['drugbank_id'] + row[
            'uniprot_id'] +  """\tex:hasDrug\t<http://example/Drug/""" + row['drugbank_id'] + """> .\n"""
        
        graph_ttl = graph_ttl + """dpi:""" + row['drugbank_id'] + row[
            'uniprot_id'] + """\tex:hasCategory\tex:""" + row['category'] + """ .\n"""
        graph_ttl = graph_ttl + """dpi:""" + row['drugbank_id'] + row[
            'uniprot_id'] + """\tex:hasGeneID\tex:""" + str(row['entrez_gene_id']) + """ .\n"""
        graph_ttl = graph_ttl + """dpi:""" + row['drugbank_id'] + row[
            'uniprot_id'] + '\tex:hasOrganism\t"' + row['organism'] + '"@en .\n'
        graph_ttl = graph_ttl + """dpi:""" + row['drugbank_id'] + row[
            'uniprot_id'] + '\tex:hasKnownAction\t"' + row['known_action'] + '"@en .\n'
        if type(row['pubmed_ids'])!=float:
            graph_ttl = graph_ttl + """dpi:""" + row['drugbank_id'] + row[
                'uniprot_id'] + '\tex:hasPubmedID\t"' + str(row['pubmed_ids']) + '"@en .\n'
        save_ttl_file(graph_ttl, 'PK_DDI_DPI.ttl')
    return set_drugs, set_protein

def save_ttl_file(graph_ttl, name):
    with open(name, 'a') as file:
        file.write(graph_ttl)

In [3]:
path = '../store_data/drug/'
df1 = load_dataset_ddi(path + 'Symmetric_DDI_corpus.csv', 'Effect')
df1.drop(columns=['Adverse events'], inplace=True)
df1.rename(columns={'Drug1ID': 'EffectorDrugID', 'Drug2ID': 'AffectedDrugID', 'Drug1Label': 'EffectorDrugLabel',
                    'Drug2Label': 'AffectedDrugLabel'}, inplace=True)
display(df1.shape, df1.head(2))

(302, 10)

,Interaction,EffectorDrugID,AffectedDrugID,EffectorDrugLabel,AffectedDrugLabel,Effect,Impact,CUI_1,CUI_2,CUI_AdverseEvent
0,The absorption of Pravastatin can be decreased...,DB00175,DB06792,Pravastatin,Lanthanum carbonate,absorption,decrease,C0085542,C0023031,NaN
1,The absorption of Levodopa can be decreased wh...,DB01235,DB00209,Levodopa,Trospium,absorption,decrease,C0023570,C0772089,NaN


In [4]:
df2 = load_dataset_ddi(path + 'Unsymmetric_DDI_corpus.csv', 'Adverse events')
df2.drop(columns=['Effect'], inplace=True)
df2.rename(columns={'Adverse events': 'Effect'}, inplace=True)
display(df2.shape, df2.head(2))

(720429, 10)

,Interaction,EffectorDrugID,AffectedDrugID,EffectorDrugLabel,AffectedDrugLabel,Effect,Impact,CUI_1,CUI_2,CUI_AdverseEvent
0,The metabolism of Caffeine can be increased wh...,DB00005,DB00201,Etanercept,Caffeine,metabolism,increase,C0717758,C0006644,C0025519
1,The metabolism of Fluvoxamine can be increased...,DB00005,DB00176,Etanercept,Fluvoxamine,metabolism,increase,C0717758,C0085228,C0025519


In [5]:
pk_ddi = pd.concat([df1, df2])
display(pk_ddi.shape, pk_ddi.head(2))

(720731, 10)

,Interaction,EffectorDrugID,AffectedDrugID,EffectorDrugLabel,AffectedDrugLabel,Effect,Impact,CUI_1,CUI_2,CUI_AdverseEvent
0,The absorption of Pravastatin can be decreased...,DB00175,DB06792,Pravastatin,Lanthanum carbonate,absorption,decrease,C0085542,C0023031,NaN
1,The absorption of Levodopa can be decreased wh...,DB01235,DB00209,Levodopa,Trospium,absorption,decrease,C0023570,C0772089,NaN


In [6]:
path = 'dataset/'
pk_ddi.to_csv(path + 'PharmacokineticDDI.csv', index=None)

In [7]:
dpi = pd.read_csv(path+'DrugsAndProteins.csv', delimiter=",")
display(dpi.shape, dpi.head(2))

(19906, 8)

,Drugbank_id,Category,Uniprot_id,Entrez_gene_id,Organism,Known_action,Actions,Pubmed_ids
0,DB00001,target,P00734,2147,Human,yes,inhibitor,10505536|10912644|11055889|11467439|11807012|1...
1,DB00006,target,P00734,2147,Human,yes,inhibitor,11060732|11504570|11833835|11923794|11929334|1...


In [8]:
cui_db = pd.read_csv('../store_data/CUI_DrugBank.csv', delimiter=",")
f1 = pd.read_csv('../store_data/drug/Symmetric_DDI_corpus.csv', delimiter=",")
f2 = pd.read_csv('../store_data/drug/Unsymmetric_DDI_corpus.csv', delimiter=",")

display(cui_db.shape, cui_db.head(2))
display(f1.head(1), f2.head(1))

(8395, 3)

,CUI_ID,DrugBankID,DrugName
0,C0772394,DB00001,lepirudin
1,C0919146,DB00001,lepirudin


,Interaction,Drug1ID,Drug2ID,Drug1Label,Drug2Label,Effect,Adverse events,Impact,CUI_1,CUI_2,CUI_AdverseEvent
0,The risk or severity of bleeding and hemorrhag...,DB01254,DB00001,Dasatinib,Lepirudin,risk or severity of,Bleeding,Increase,C1455147,C0772394,C0019080


,Interaction,EffectorDrugID,AffectedDrugID,EffectorDrugLabel,AffectedDrugLabel,Adverse events,Effect,Impact,CUI_1,CUI_2,CUI_AdverseEvent
0,Apixaban may increase the anticoagulant activi...,DB06605,DB00001,Apixaban,Lepirudin,Anticoagulant_activities,NaN,Increase,C1831808,C0772394,C0003280


In [9]:
df1 = pk_ddi[['EffectorDrugID', 'EffectorDrugLabel', 'CUI_1']]
df1.drop_duplicates(keep='first', inplace=True, ignore_index=True)
df2 = pk_ddi[['AffectedDrugID', 'AffectedDrugLabel', 'CUI_2']]
df2.drop_duplicates(keep='first', inplace=True, ignore_index=True)
df1.rename(columns={'EffectorDrugID': 'DrugID', 'EffectorDrugLabel': 'DrugName',
                                   'CUI_1': 'DrugCUI'}, inplace=True)
df2.rename(columns={'AffectedDrugID': 'DrugID', 'AffectedDrugLabel': 'DrugName',
                                   'CUI_2': 'DrugCUI'}, inplace=True)

df = pd.concat([df1, df2])
df.drop_duplicates(keep='first', inplace=True, ignore_index=True)
#df1.to_csv('Drugs.csv', index=None)
df

C:\Users\rivasa\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\rivasa\Anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,DrugID,DrugName,DrugCUI
0,DB00175,Pravastatin,C0085542
1,DB01235,Levodopa,C0023570
2,DB09073,Palbociclib,C3853822
3,DB00227,Lovastatin,C0024027
4,DB00253,Medrysone,C0065865
...,...,...,...
2880,DB12377,Relebactam,C4317244
2881,DB12989,Neosaxitoxin,C0068533
2882,DB13153,Levomenol,C0053774
2883,DB13324,Tetrazepam,C0076341


In [10]:
missing = set(dpi.Drugbank_id.unique()) - set(df.DrugID.unique())
print('missing: ', len(missing))
df1 = cui_db.loc[cui_db.DrugBankID.isin(missing)]

df1.drop_duplicates(['DrugBankID', 'DrugName'],keep='first', inplace=True, ignore_index=True)
df1.rename(columns={'CUI_ID': 'DrugCUI', 'DrugBankID': 'DrugID'}, inplace=True)
add_df = df1.reindex(columns=['DrugID','DrugName', 'DrugCUI'])
add_df['DrugName'] = add_df['DrugName'].str.capitalize()
add_df

missing:  4377


C:\Users\rivasa\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\rivasa\Anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,DrugID,DrugName,DrugCUI
0,DB00001,Lepirudin,C0772394
1,DB00002,Cetuximab,C0995188
2,DB00004,Denileukin diftitox,C0717670
3,DB00006,Bivalirudin,C0168273
4,DB00010,Sermorelin,C0142046
...,...,...,...
1239,DB08927,Amperozide,C0051747
1240,DB08936,Chlorcyclizine,C0008183
1241,DB09014,Captodiame,C0301409
1242,DB09018,Bromopride,C0054129


In [11]:
df1 = f1[['Drug1ID', 'Drug1Label', 'CUI_1']]
df1.drop_duplicates(keep='first', inplace=True, ignore_index=True)
df2 = f1[['Drug2ID', 'Drug2Label', 'CUI_2']]
df2.drop_duplicates(keep='first', inplace=True, ignore_index=True)
df1.rename(columns={'Drug1ID': 'DrugID', 'Drug1Label': 'DrugName',
                                   'CUI_1': 'DrugCUI'}, inplace=True)
df2.rename(columns={'Drug2ID': 'DrugID', 'Drug2Label': 'DrugName',
                                   'CUI_2': 'DrugCUI'}, inplace=True)

df1 = pd.concat([df1, df2])
df1.drop_duplicates(keep='first', inplace=True, ignore_index=True)
df1 = df1.loc[df1.DrugID.isin(missing)]
add_df1 = df1.reindex(columns=['DrugID','DrugName', 'DrugCUI'])
add_df1

C:\Users\rivasa\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\rivasa\Anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,DrugID,DrugName,DrugCUI
1,DB00001,Lepirudin,C0772394
116,DB00054,Abciximab,C0288672
117,DB00063,Eptifibatide,C0253563
123,DB01088,Iloprost,C0079594
126,DB01207,Ridogrel,C0072926
...,...,...,...
3530,DB02285,Protoporphyrin,C0033733
3538,DB02178,Phenylacetaldehyde,C0070619
3543,DB03587,Pyruvaldehyde,C0034338
3550,DB00943,Zalcitabine,C0012132


In [12]:
df1 = f2[['EffectorDrugID', 'EffectorDrugLabel', 'CUI_1']]
df1.drop_duplicates(keep='first', inplace=True, ignore_index=True)
df2 = f2[['AffectedDrugID', 'AffectedDrugLabel', 'CUI_2']]
df2.drop_duplicates(keep='first', inplace=True, ignore_index=True)
df1.rename(columns={'EffectorDrugID': 'DrugID', 'EffectorDrugLabel': 'DrugName',
                                   'CUI_1': 'DrugCUI'}, inplace=True)
df2.rename(columns={'AffectedDrugID': 'DrugID', 'AffectedDrugLabel': 'DrugName',
                                   'CUI_2': 'DrugCUI'}, inplace=True)

df1 = pd.concat([df1, df2])
df1.drop_duplicates(keep='first', inplace=True, ignore_index=True)
df1 = df1.loc[df1.DrugID.isin(missing)]
add_df2 = df1.reindex(columns=['DrugID','DrugName', 'DrugCUI'])
add_df2

C:\Users\rivasa\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\rivasa\Anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,DrugID,DrugName,DrugCUI
2,DB00001,Lepirudin,C0772394
4,DB00013,Urokinase,C0042071
10,DB00686,Pentosan polysulfate,C0030888
95,DB03404,Hemin,C0018988
185,DB00002,Cetuximab,C0995188
...,...,...,...
4032,DB02753,Selenoinosine,NaN
4035,DB03049,S-Selanyl Cysteine,C0006992
4036,DB03070,Selenazole-4-carboxyamide-adenine dinucleotide,C0251080
4038,DB03423,S-Adenosyl-L-Homoselenocysteine,C0036002


In [13]:
df1 = pd.concat([add_df, add_df1, add_df2])
df1.drop_duplicates(keep='first', inplace=True, ignore_index=True)
df1 = pd.concat([df1, df])
df1

,DrugID,DrugName,DrugCUI
0,DB00001,Lepirudin,C0772394
1,DB00002,Cetuximab,C0995188
2,DB00004,Denileukin diftitox,C0717670
3,DB00006,Bivalirudin,C0168273
4,DB00010,Sermorelin,C0142046
...,...,...,...
2880,DB12377,Relebactam,C4317244
2881,DB12989,Neosaxitoxin,C0068533
2882,DB13153,Levomenol,C0053774
2883,DB13324,Tetrazepam,C0076341


In [14]:
m = set(dpi.Drugbank_id.unique()) - set(df1.DrugID.unique())
print('missing: ', len(m))

add_df3 = pd.DataFrame(columns=['DrugID', 'DrugName', 'DrugCUI'])
add_df3['DrugID'] = list(m)

df1 = pd.concat([df1, add_df3])
df1.drop_duplicates(['DrugID'],keep='first', inplace=True, ignore_index=True)
df1

missing:  3124


,DrugID,DrugName,DrugCUI
0,DB00001,Lepirudin,C0772394
1,DB00002,Cetuximab,C0995188
2,DB00004,Denileukin diftitox,C0717670
3,DB00006,Bivalirudin,C0168273
4,DB00010,Sermorelin,C0142046
...,...,...,...
7257,DB06993,NaN,NaN
7258,DB02758,NaN,NaN
7259,DB03982,NaN,NaN
7260,DB07838,NaN,NaN


In [18]:
df1.to_csv('Drugs.csv', index=None)

In [17]:
len(df1.DrugID.unique())

7262

In [20]:
df2 = dpi[['Pubmed_ids']]
dpi_pubmed = pd.DataFrame()
for index, row in dpi.iterrows():
    if type(row['Pubmed_ids'])!=float:
        df3 = pd.DataFrame(data=row['Pubmed_ids'].split('|'), columns=['Pubmed_ids'])
        df3['Drugbank_id'] = row['Drugbank_id']
        df3['Uniprot_id'] = row['Uniprot_id']
        dpi_pubmed = pd.concat([dpi_pubmed, df3])
    
dpi_pubmed

,Pubmed_ids,Drugbank_id,Uniprot_id
0,10505536,DB00001,P00734
1,10912644,DB00001,P00734
2,11055889,DB00001,P00734
3,11467439,DB00001,P00734
4,11807012,DB00001,P00734
...,...,...,...
0,17591945,DB08889,P40306
0,23090647,DB08890,P25092
0,23590413,DB08907,P13866
0,23056222,DB08908,Q14145


In [21]:
dpi_pubmed.to_csv('dpi_pubmed.csv', index=None)

In [4]:
set_drugs = get_ddi_triples(pk_ddi)
set_drugs, set_protein = get_dpi_triples(dpi, set_drugs)

In [5]:
from collections import OrderedDict

In [6]:
def save_ttl_file(graph_ttl):
    #open text file
    ttl_file = open("PK_DDI_DPI_00.ttl", "w")
    #write string to file
    n = ttl_file.write(graph_ttl)
    #close file
    ttl_file.close()

In [7]:
graph_ttl = open("PK_DDI_DPI.ttl").read()
graph_ttl = "\n".join(list(OrderedDict.fromkeys(graph_ttl.split("\n"))))
save_ttl_file(graph_ttl)

In [9]:
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib.plugins.sparql.processor import SPARQLResult

In [10]:
dbpedia = SPARQLWrapper('https://dbpedia.org/sparql/')

In [48]:
drug = set(pk_ddi.precipitantDrug.unique())
d = set(pk_ddi.objectDrug.unique())
drug.update(d)
d = set(dpi.drugbank_id.unique())
drug.update(d)
len(drug)

6693

In [79]:

name = """','""".join(drug)
name = "'"+name+"'"


In [82]:
df1

,db_id,relation,dbpedia_resource
0,DB03974,dbo:drugbank,NaN
1,DB00503,dbo:drugbank,http://dbpedia.org/resource/Ritonavir
2,DB13264,dbo:drugbank,NaN
3,DB00588,dbo:drugbank,http://dbpedia.org/resource/Fluticasone
4,DB07829,dbo:drugbank,NaN
...,...,...,...
6688,DB00733,dbo:drugbank,NaN
6689,DB02272,dbo:drugbank,NaN
6690,DB02028,dbo:drugbank,NaN
6691,DB09185,dbo:drugbank,NaN


In [84]:
df1 = pd.DataFrame(data=drug, columns=['db_id'])
df1['relation'] = 'dbo:drugbank'

for index, row in df1.iterrows():
    query = """select distinct ?s where {
?s dbo:drugbank '""" + row['db_id'] + """' .}"""
    dbpedia.setQuery(query)
    dbpedia.setReturnFormat(JSON)
    results = dbpedia.query().convert()
    if len(results['results']['bindings'])>0:
        df1.at[index, 'dbpedia_resource'] = results['results']['bindings'][0]['s']['value']

df1

,db_id,relation,dbpedia_resource
0,DB03974,dbo:drugbank,NaN
1,DB00503,dbo:drugbank,http://dbpedia.org/resource/Ritonavir
2,DB13264,dbo:drugbank,NaN
3,DB00588,dbo:drugbank,http://dbpedia.org/resource/Fluticasone
4,DB07829,dbo:drugbank,NaN
...,...,...,...
6688,DB00733,dbo:drugbank,http://dbpedia.org/resource/Pralidoxime
6689,DB02272,dbo:drugbank,NaN
6690,DB02028,dbo:drugbank,NaN
6691,DB09185,dbo:drugbank,http://dbpedia.org/resource/Viloxazine


In [86]:
len(df1.dbpedia_resource.unique())

1765